In [ ]:
from jobflow import job, Flow

In [ ]:
from pyiron_base import Project

In [ ]:
from python_workflow_definition.jobflow import write_workflow_json

In [ ]:
from python_workflow_definition.pyiron_base import load_workflow_json

In [ ]:
from quantum_espresso_workflow import (
    calculate_qe as _calculate_qe, 
    generate_structures as _generate_structures, 
    get_bulk_structure as _get_bulk_structure, 
    plot_energy_volume_curve as _plot_energy_volume_curve,
)

In [ ]:
calculate_qe = job(_calculate_qe, data=["energy", "volume", "structure"])
generate_structures = job(_generate_structures, data=["i" for i in range(100)])
plot_energy_volume_curve = job(_plot_energy_volume_curve)
get_bulk_structure = job(_get_bulk_structure)

In [ ]:
pseudopotentials = {"Al": "Al.pbe-n-kjpaw_psl.1.0.0.UPF"}

In [ ]:
structure = get_bulk_structure(
    name="Al",
    a=4.05,
    cubic=True,
)

In [ ]:
calc_mini = calculate_qe(
    working_directory="mini",
    input_dict={
        "structure": structure.output,
        "pseudopotentials": pseudopotentials,
        "kpts": (3, 3, 3),
        "calculation": "vc-relax",
        "smearing": 0.02,
    },
)

In [ ]:
number_of_strains = 5
structure_lst = generate_structures(
    structure=calc_mini.output.structure,
    strain_lst=np.linspace(0.9, 1.1, number_of_strains),
)

In [ ]:
job_strain_lst = []
for i in range(number_of_strains):
    calc_strain = calculate_qe(
        working_directory="strain_" + str(i),
        input_dict={
            "structure": getattr(structure_lst.output, str(i)),
            "pseudopotentials": pseudopotentials,
            "kpts": (3, 3, 3),
            "calculation": "scf",
            "smearing": 0.02,
        },
    )
    job_strain_lst.append(calc_strain)

In [ ]:
plot = plot_energy_volume_curve(
    volume_lst=[job.output.volume for job in job_strain_lst],
    energy_lst=[job.output.energy for job in job_strain_lst],
)

In [ ]:
flow = Flow([structure, calc_mini, structure_lst] + job_strain_lst + [plot])

In [ ]:
write_workflow_json(flow=flow, file_name="workflow_jobflow.json")

In [ ]:
!cat workflow_jobflow.json

In [ ]:
pr = Project("test")
pr.remove_jobs(recursive=True, silently=True)

In [ ]:
delayed_object = load_workflow_json(project=pr, file_name="workflow_jobflow.json")
delayed_object.draw()

In [ ]:
delayed_object.pull()